In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import seaborn as sns
from matplotlib import style
import math
import random

In [2]:
#LP MAXIMIZATION MODEL

from pulp import *
Promo_Model = LpProblem("Pharma_Promotion_Model", LpMaximize)

In [3]:
#PARAMETERS

T=12 # end of promotion horizon
PHorizon = list(range(0, T))

I=10 # product count 
Products = list(range(0, I))

#NOT USED RIGHT NOW
Promo_Levels = [0,1,2]

Demand = np.zeros((I, T), dtype = int)

for i in Products:
    for t in PHorizon:
       Demand[i][t] = random.randint(0, 10000)
    
Total_PaidQty_Limit = 1000000
Total_FreeQty_Limit = 250000

Demand

array([[8435, 4622, 2644,  881, 8355,  819, 1094, 6801, 4308, 5114, 8125,
        8372],
       [1955, 4818, 8335, 5790, 8452, 4804, 7342,  935,  643, 2383, 5798,
        4187],
       [3891, 8663, 3202, 4225, 5962, 9826, 4124, 1007,  260, 2705, 2865,
        2074],
       [7874,  629, 4853, 5420, 4453, 3986, 4841, 3278, 3662, 3113, 6058,
        6997],
       [ 319, 9736, 7238, 6743, 8499,  609, 2291, 4391, 5281, 9401, 1008,
        6502],
       [5503, 3540, 2635,  422, 7912, 5447, 6987, 3623, 7525, 9582, 8731,
        1630],
       [4361, 7074, 4827, 5489, 8941, 8158, 5971, 9812, 4982, 8760, 3322,
        7009],
       [5680, 2309, 6097, 6793, 1912, 9376,  393, 9665, 4594, 1834, 8148,
        1113],
       [3596, 5811, 4156, 6204, 6996, 4361, 9208, 7318, 7225, 1203, 3580,
        4496],
       [9048, 2091, 4958, 1478, 6344, 3292, 7751, 4133, 9551, 4757, 9170,
        9296]])

In [4]:
#DECISION VARIABLES

#Paid Quantity
PaidQtyLimit = 10000
P = LpVariable.dicts("Paid_Quantity",(Products,PHorizon),lowBound=0, upBound=PaidQtyLimit, cat='Integer')

#Free Quantity
FreeQtyLimit = 5000
F = LpVariable.dicts("Free_Quantity",(Products,PHorizon),lowBound=0, upBound=FreeQtyLimit, cat='Integer')

#Promo Ratio
#R = LpVariable.dicts("Promo_Ratio",(Products,PHorizon),lowBound=0, upBound=1, cat='Continuous')

#Is Product "i" at Time "t" Promoted?
X = LpVariable.dicts("IsPromoted",(Products,PHorizon),cat='Binary')

In [5]:
#OBJECTIVE FUNCTION

Total_Paid_Quantity = lpSum(lpSum(P[i][t] for i in Products) for t in PHorizon)

Promo_Model += Total_Paid_Quantity

In [6]:
#DEMAND SATISFACTION
for i in Products:
    for t in PHorizon:
        Promo_Model += P[i][t] + F[i][t] == Demand[i][t]*(1 + F[i][t]*(1/FreeQtyLimit))
    
#NO FREE WITHOUT PROMOTION
M = FreeQtyLimit
for i in Products:
    for t in PHorizon:
        Promo_Model += F[i][t] <= X[i][t]*M
        Promo_Model += F[i][t] >= X[i][t]-0.5
    
#PROMO RATIO
#for i in Products:
#    for t in PHorizon:
#        Promo_Model += F[i][t] == R[i][t]*Demand[i][t]      

#PROMOTED PRODUCTS AT TIME "t"
for t in PHorizon:
    Promo_Model += lpSum(X[i][t] for i in Products) == 5
    
#PROMO PERIOD FOR PRODUCT "i"  
for i in Products:
    Promo_Model += lpSum(X[i][t] for t in PHorizon) == 6

#TOTAL PAID QUANTITY CAPACITY
for i in Products:
    for t in PHorizon:
        Promo_Model += lpSum(P[i][t] for t in PHorizon) <= Total_PaidQty_Limit
    
#TOTAL FREE QUANTITY CAPACITY
for i in Products:
    for t in PHorizon:
        Promo_Model += lpSum(F[i][t] for t in PHorizon) <= Total_FreeQty_Limit

In [7]:
solver = GUROBI()
solver.solve(Promo_Model)


# The status of the solution is printed to the screen
print("Status:", LpStatus[Promo_Model.status])


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.3.0 22D68)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 622 rows, 360 columns and 3840 nonzeros
Model fingerprint: 0x7e4d0b03
Variable types: 0 continuous, 360 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-03, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+04]
  RHS range        [5e-01, 1e+06]
Presolve removed 429 rows and 139 columns
Presolve time: 0.19s
Presolved: 193 rows, 221 columns, 542 nonzeros
Variable types: 0 continuous, 221 integer (103 binary)

Root relaxation: objective 6.753476e+05, 135 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 675347.614    0   39    

     0     0 673915.746    0   32          - 673915.746      -     -    0s
     0     0 673907.444    0   42          - 673907.444      -     -    0s
     0     0 673903.916    0   45          - 673903.916      -     -    0s
     0     0 673892.474    0   14          - 673892.474      -     -    0s
     0     0 673888.588    0   45          - 673888.588      -     -    0s
     0     0 673867.404    0   14          - 673867.404      -     -    0s
     0     0 673862.571    0   32          - 673862.571      -     -    0s
     0     0 673838.100    0   42          - 673838.100      -     -    0s
     0     0 673833.230    0   45          - 673833.230      -     -    0s
     0     0 673822.528    0   14          - 673822.528      -     -    0s
     0     0 673819.424    0   32          - 673819.424      -     -    0s
     0     0 673810.588    0   42          - 673810.588      -     -    0s
     0     0 673806.652    0   45          - 673806.652      -     -    0s
     0     0 673794.219  

H   68    20                    647343.00000 672664.328  3.91%   1.4    0s
H 1171    29                    648249.00000 672657.455  3.77%   1.0    1s
H 1201    29                    648869.00000 672657.455  3.67%   1.0    1s
H 1372    29                    653428.00000 672657.455  2.94%   1.0    1s
H18314    81                    654665.00000 672579.065  2.74%   1.0    1s
H18412    81                    654973.00000 672579.065  2.69%   1.0    1s
H22866   111                    656971.00000 672579.065  2.38%   1.0    1s
H34169   120                    657174.00000 672555.846  2.34%   1.0    1s

Cutting planes:
  MIR: 12
  StrongCG: 7
  Flow cover: 1

Explored 34799 nodes (37770 simplex iterations) in 2.45 seconds (0.84 work units)
Thread count was 8 (of 8 available processors)

Solution count 10: 657174 656971 655176 ... 645499

Optimal solution found (tolerance 1.00e-04)
Best objective 6.571740000000e+05, best bound 6.571740000000e+05, gap 0.0000%
Gurobi status= 2
Status: Optimal


In [8]:
# OUTPUT

# OPTIMIZED OBJECTIVE FUNCTION
print("Optimal Total Paid Quantity = ", pulp.value(Promo_Model.objective),"\n")

# PRINTS VARIABLES AND OPTIMAL VALUES
for v in Promo_Model.variables():
    print(v.name, "=", v.varValue)

Optimal Total Paid Quantity =  657174.0 

Free_Quantity_0_0 = 2000.0
Free_Quantity_0_1 = 2500.0
Free_Quantity_0_10 = 3000.0
Free_Quantity_0_11 = 1250.0
Free_Quantity_0_2 = 0.0
Free_Quantity_0_3 = 0.0
Free_Quantity_0_4 = 2000.0
Free_Quantity_0_5 = 0.0
Free_Quantity_0_6 = 0.0
Free_Quantity_0_7 = 5000.0
Free_Quantity_0_8 = 0.0
Free_Quantity_0_9 = 0.0
Free_Quantity_1_0 = 0.0
Free_Quantity_1_1 = 2500.0
Free_Quantity_1_10 = 0.0
Free_Quantity_1_11 = 0.0
Free_Quantity_1_2 = 2000.0
Free_Quantity_1_3 = 5000.0
Free_Quantity_1_4 = 1250.0
Free_Quantity_1_5 = 1250.0
Free_Quantity_1_6 = 5000.0
Free_Quantity_1_7 = 0.0
Free_Quantity_1_8 = 0.0
Free_Quantity_1_9 = 0.0
Free_Quantity_2_0 = 5000.0
Free_Quantity_2_1 = 0.0
Free_Quantity_2_10 = 0.0
Free_Quantity_2_11 = 0.0
Free_Quantity_2_2 = 0.0
Free_Quantity_2_3 = 200.0
Free_Quantity_2_4 = 5000.0
Free_Quantity_2_5 = 0.0
Free_Quantity_2_6 = 1250.0
Free_Quantity_2_7 = 0.0
Free_Quantity_2_8 = 250.0
Free_Quantity_2_9 = 1000.0
Free_Quantity_3_0 = 0.0
Free_Quantit

In [9]:
#PROMO RATIO IS CALCULATED BASED ON THE OPTIMUM P[i][t] AND F[i][t] VALUES
Promo_Ratio = np.zeros((I, T), dtype = float)

for i in Products:
    for t in PHorizon:
        Promo_Ratio[i][t] = (F[i][t].varValue)/(P[i][t].varValue + F[i][t].varValue)

df_Promo_Ratio = pd.DataFrame(Promo_Ratio,
                 index=range(1,I+1),
                 columns=range(1,T+1))
df_Promo_Ratio

,1,2,3,4,5,6,7,8,9,10,11,12
1,0.169362,0.360594,0.000000,0.000000,0.170984,0.000000,0.000000,0.367593,0.000000,0.000000,0.230769,0.119446
2,0.000000,0.345925,0.171394,0.431779,0.118315,0.208160,0.340507,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.642508,0.000000,0.000000,0.045517,0.419322,0.000000,0.242483,0.000000,0.915751,0.308071,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.418130,0.516422,0.508440,0.000000,0.803084,0.412677,0.357296
5,0.000000,0.000000,0.345399,0.370755,0.000000,0.000000,0.000000,0.569346,0.473395,0.000000,0.551146,0.384497
6,0.454298,0.067259,0.000000,0.000000,0.243058,0.458968,0.357807,0.000000,0.325446,0.000000,0.000000,0.000000
7,0.573263,0.353407,0.000000,0.000000,0.000000,0.204298,0.418690,0.000000,0.000000,0.140002,0.000000,0.356684
8,0.440141,0.000000,0.410038,0.368026,0.000000,0.000000,0.000000,0.000000,0.362792,0.908760,0.204549,0.000000
9,0.000000,0.430219,0.240616,0.402966,0.000000,0.573263,0.000000,0.341623,0.346021,0.000000,0.000000,0.000000
10,0.000000,0.000000,0.336157,0.000000,0.394073,0.000000,0.000000,0.604887,0.000000,0.525541,0.049569,0.059763
